In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/hr-analytics-job-change-of-data-scientists/aug_train.csv')

In [ ]:
train.head()

# Understanding DATA

In [ ]:
train.shape

In [ ]:
import seaborn as sns
sns.heatmap(train.isnull())

Gender , Major Discipline and Company size and company type have a lot of missing values, rest are good

In [ ]:
percent_missing = train.isnull().sum() * 100 / len(train)
missing_value_df = pd.DataFrame({'column_name': train.columns,
                                 'percent_missing': percent_missing})

In [ ]:
missing_value_df

let us explore different types of values in column

In [ ]:
train

In [ ]:
sns.countplot(train['target'])

So , probablity is higher that a person will not change job

In [ ]:
sns.countplot(train['relevent_experience'],hue=train['target'])

> Conclusion 

1.Most of the candidates has relevant experience 

2.Candidates with relevant experience are not looking for a job change

3.Candidates with no relevant experience are more keen to change job

In [ ]:
sns.countplot(train['enrolled_university'],hue=train['target'])

Conclusion :

1.Most of the person are having no enrollment in any university and those without any enrollment have no interest in changing their job 

2.Part Time course enrolled students are very less and they also dont want to change

3.Full time enrolled are not very much also but in proportion they have higher chance of changing the job than others

In [ ]:
sns.boxplot(x='target',y='city_development_index',data=train)

> Conclusion

1.Most of the people who are not changing jobs are from city with high development index so basically they are having a comfortable life in respective city are not willing to change jobs

2.People in cities with less development index tend to change their jobs for better life style maybe!

In [ ]:
sns.boxplot(y='training_hours',x='target',data=train)

In [ ]:
sns.barplot(y='training_hours',x='target',data=train)

There is no much difference between training hours of those who are wishing to change and those who are not , so not much can be deduced from this classification

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(20,6))
sns.countplot(train['experience'],hue=train['target'])

> Conclusion : 

1.people who have experience less than 1 year have more tendency to change their job while those with more than 20 years of experience have very less tendency of changing jobz

 2.As experience increase , tendency to change the job becomes more and more less

In [ ]:
sns.countplot(train['education_level'],hue=train['target'])

> conclusion

1.Graduates have very less chance of leaving 

2.Regardless of any education level , there is a very less chance of changing job 

In [ ]:
sns.countplot(train['major_discipline'],hue=train['target'])

> Conclusion

1. Mostly people are STEM

2. In STEM , people tend not to change the job  

3. People not tend to change job in any field 

4. There is a good proportion of people relatively in Business Degree that tend to change job

# DATA PREPARING FOR MODEL

First I would drop all the rows 

In [ ]:
print('shape before dropping values',train.shape)

In [ ]:
train = train.drop(['company_type','company_size','major_discipline','gender'],axis=1)

i dropped all those columns which had more than 10 percent of missing data

In [ ]:
train = train.dropna()

Removing all rows which were having so missing values

In [ ]:
print('Shape after data cleaning',train.shape)

# DATA TRANSFORMATION

In [ ]:
train.head()

Removing all the extra columns : 

1.enrolee_id

2. city as it is represented by development index

Encoding Categorical value

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train['relevent_experience'] = le.fit_transform(train['relevent_experience'])

In [ ]:
train['enrolled_university'] = le.fit_transform(train['enrolled_university'])
train['education_level'] = le.fit_transform(train['education_level'])
train['experience'] = le.fit_transform(train['experience'])
train['last_new_job'] = le.fit_transform(train['last_new_job'])

In [ ]:
train.head(2)

In [ ]:
train_resample = train.copy()

In [ ]:
train = train.drop(['enrollee_id','city'],axis=1)

as we have seen earlier, this dataset is very uneven , so we would consider both ways to create model , but first with original data

# REMOVING OUTLIERS

In [ ]:
from scipy import stats
import numpy as np
z = np.abs(stats.zscore(train))


In [ ]:
threshold = 3
train = train[(z < 3).all(axis=1)]

In [ ]:
train.shape

# Normalizing our DATA

In [ ]:
#breaking into training and testing dataset
x = train[['city_development_index', 'relevent_experience', 'enrolled_university',
       'education_level', 'experience', 'last_new_job', 'training_hours']]
y = train['target']
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.91,stratify=y)

In [ ]:
from sklearn import preprocessing

x_train = preprocessing.normalize(x_train)
x_test = preprocessing.normalize(x_test)

# MODEL

Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
linear = LogisticRegression()
linear.fit(x_train,y_train)
print('Accuracy on test set is : ',linear.score(x_test,y_test))

Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier()
tree.fit(x_train,y_train)
print('Accuracy on test set is ', tree.score(x_test,y_test))

SVM

In [ ]:
from sklearn.svm import SVC
svc = SVC()
svc.fit(x_train,y_train)
print('Accuracy on test set is ', svc.score(x_test,y_test))


BOOSTING

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
boost = GradientBoostingClassifier(n_estimators=200)
boost.fit(x_train,y_train)
print('accuracy on test set : ',boost.score(x_test,y_test))

RANDOM FOREST

In [ ]:
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier()
forest.fit(x_train,y_train)
print('accuracy on test set : ',forest.score(x_test,y_test))


SVC with different Parameters

In [ ]:
svc = SVC(kernel='poly',gamma=5,C=10,degree=5)
svc.fit(x_train,y_train)
svc.score(x_test,y_test)

KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=100)
knn.fit(x_train,y_train)
knn.score(x_test,y_test)

STACKING ALGORITHMS

In [ ]:
# define the base models
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
level0 = list()
level0.append(('lr', LogisticRegression()))
level0.append(('knn', KNeighborsClassifier()))
level0.append(('cart', DecisionTreeClassifier()))
level0.append(('svm', SVC()))
level0.append(('forest',RandomForestClassifier()))

In [ ]:
#define meta model
from sklearn.ensemble import StackingClassifier
# define meta learner model
level1 = LogisticRegression()
# define the stacking ensemble
model = StackingClassifier(estimators=level0, final_estimator=level1, cv=5)

In [ ]:
#running model
model.fit(x_train,y_train)

In [ ]:
#Checking Accuracy
model.score(x_test,y_test)

to read more about stacking in ML
https://www.kaggle.com/cosmosankur/stacking-in-machine-learning#Making-Final-Model-For-Stacking

Let us do some resampling on our data as this is imbalanced data

# 1. OVERSAMPLING

Oversampling can be defined as adding more copies of the minority class. Oversampling can be a good choice when you don’t have a ton of data to work with.

In [ ]:
train_resample.head()

made this copy previously to use it now :)

In [ ]:
train_resample = train_resample.drop(['enrollee_id','city'],axis=1)

In [ ]:
x = train_resample[['city_development_index', 'relevent_experience', 'enrolled_university',
       'education_level', 'experience', 'last_new_job', 'training_hours']]
y = train_resample['target']
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.91,stratify=y)

In [ ]:
from sklearn.utils import resample
# concatenate our training data back together
X = pd.concat([x_train, y_train], axis=1)
# separate minority and majority classes
not_change = X[X['target']==0]
change = X[X['target']==1]

# upsample minority
change_upsampled = resample(change,
                          replace=True, # sample with replacement
                          n_samples=len(not_change), # match number in majority class
                          random_state=27) # reproducible results
# combine majority and upsampled minority
upsampled = pd.concat([not_change, change_upsampled])

In [ ]:
upsampled['target'].value_counts()

In [ ]:
y_train = upsampled['target']
X_train = upsampled.drop('target', axis=1)

In [ ]:
#Logistic Regression
linear.fit(X_train,y_train)
print('Accuracy on test set after oversampling',linear.score(x_test,y_test))

In [ ]:
#Boosting
boost.fit(X_train,y_train)
print('Accuracy on test set after oversampling',boost.score(x_test,y_test))

In [ ]:
#Decision Tree
linear.fit(X_train,y_train)
print('Accuracy on test set after oversampling',tree.score(x_test,y_test))

# UNDERSAMPLING

Undersampling can be defined as removing some observations of the majority class. Undersampling can be a good choice when you have a ton of data -think millions of rows. But a drawback is that we are removing information that may be valuable. This could lead to underfitting and poor generalization to the test set.

In [ ]:
# still using our separated classes fraud and not_fraud from above

# downsample majority
not_change_downsampled = resample(not_change,
                                replace = False, # sample without replacement
                                n_samples = len(change), # match minority n
                                random_state = 27) # reproducible results

# combine minority and downsampled majority
downsampled = pd.concat([not_change_downsampled, change])

In [ ]:
downsampled['target'].value_counts()

In [ ]:
y_train = downsampled['target']
X_train = downsampled.drop('target', axis=1)

In [ ]:
#Logistic Regression
linear.fit(X_train,y_train)
print('Accuracy on test set after undersampling',linear.score(x_test,y_test))

In [ ]:
#BOOSTING
boost.fit(X_train,y_train)
print('Accuracy on test set after undersampling',boost.score(x_test,y_test))

since sampling has not affected much of our accuracy , original data was giving better accuracy

THANKS FOR VIEWING THIS NOTEBOOK